# 0. Import necessary packages

In [ ]:
import numpy as np
import pandas as pd
import requests
import os
import json

from rdkit import Chem
from tqdm import tqdm
from thermo import functional_groups
from Bio import Entrez
from chembl_structure_pipeline import checker

from rdkit.Chem import rdMolDescriptors, Descriptors, Lipinski, Crippen, inchi
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from concurrent.futures import ThreadPoolExecutor, as_completed

[00:03:59] Initializing Normalizer


In [ ]:
data_folder = 'H:/coding/HiChem/curation/data/AID_1798_M1_Muscarinic_Agonist'

# 1. Data gathering

Before importing data, need to identify which AIDs will be included. Data will be imported from https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid={AID}. For more information on PubChem's programmatic access, refer to: https://pubchem.ncbi.nlm.nih.gov/docs/bioassays. Some other programmatic access options available such as PUG-REST. However, these might not be optimal for bulk retrieval or handling of large dataset due to the limitation of request volume.

Data for individual assays include 7 required columns (CIDs, isomeric SMILES, etc.) and optional test results. Refer to https://ftp.ncbi.nlm.nih.gov/pubchem/Bioassay/CSV/README for further details. For datasets intended for regression model, additional columns could be extracted accordingly.

In [ ]:
# Desired AIDs:
AIDs = [626, 1488, 1741]

#Keep unique values in list AIDs (since there could be overlapping AIDs from different targets or project)
AIDs = list(set(AIDs))
AIDs = [str(AID) for AID in AIDs]
print('Number of datasets retrieving: ', len(AIDs))

Number of datasets retrieving:  3


In [ ]:
#Data to be extracted from the assay (modified by users' choice)
col_list = ['PUBCHEM_CID','PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_OUTCOME']

count = 0
for AID in AIDs:
    url = f'https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid={AID}'
    assay = pd.read_csv(url, usecols=col_list)

    #convert SMILES to string
    assay['PUBCHEM_EXT_DATASOURCE_SMILES'] = assay['PUBCHEM_EXT_DATASOURCE_SMILES'].astype(str)

    #delete rows with nan values
    assay = assay.dropna(subset=['PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_OUTCOME', 'PUBCHEM_CID'])

    #convert cids to int:
    assay['PUBCHEM_CID'] = assay['PUBCHEM_CID'].astype(int)

    #reindex assay dataframe from 0:
    assay.reset_index(drop=True, inplace=True)

    #Create a new folder to save the data:
    if not os.path.exists(f'{data_folder}/before_finished/step_1'):
        os.makedirs(f'{data_folder}/before_finished/step_1')

    #save assay data
    assay.to_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv', index=False)
    count += 1
    print(f'Completed {count} out of {len(AIDs)} datasets.')

1 out of 3 complete
2 out of 3 complete
3 out of 3 complete


# 2. Isomeric SMILES

For the purpose of our project, we would like to include isomeric form of SMILES representation in our final dataset. Although PubChem claimed that their datatable should include isomeric SMILES (https://pubchem.ncbi.nlm.nih.gov/docs/bioassays), some dataset might include non-isomeric SMILES. This step is to import isomeric SMILES based on CIDs.

Several packages such as RDkit have modules to return isomeric SMILES from a given input SMILES. However, for consistency, we decided to use the PubChem Identifier Exchange Service, which take an input identifier (CIDs, SMILES, InChI, etc.)  and return the corresponding identifier (CIDs, isomeric SMILES, InChIs, etc.). Here, we export the list of CIDs for compounds in our dataset and use this server to retrieve their isomeric SMILES. For more information, refer to: https://pubchem.ncbi.nlm.nih.gov/docs/identifier-exchange-service

In [ ]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_2'):
    os.makedirs(f'{data_folder}/rabebefore_finishedfore_finishedw/step_2')

#Export list of CIDs to csv with one column without the column name:
for AID in AIDs:
    assay = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv')
    assay['PUBCHEM_CID'].to_csv(f'{data_folder}/before_finished/step_2/CID{AID}.csv', index=False, header=False)

#After this step, we submit the list at https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi with operator type "same CID" and Output IDs "SMILES" (isomeric SMILES by default)
#https://pubchem.ncbi.nlm.nih.gov/docs/identifier-exchange-service for more details
#Here I named the output file as "SMILES{AID}.txt"

In [ ]:
def check_isomeric_smiles(AIDs):
    """
    Check if the SMILES in the assay are isomeric.
    Input:
        AIDs (list of str)
    Output:
        non_isomeric_smi_cids (dictionary: AID -> list of non-isomeric CIDs)
    """
    non_isomeric_smi_cids = {}
    for AID in AIDs:
        non_isomeric_smi_cids[AID] = []
        #import SMILES.txt file as a table:
        correct_isomeric_smiles = pd.read_csv(f'{data_folder}/before_finished/step_2/isomeric_smi_{AID}.txt', sep='\t', header=None)
        assay = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv')

        #compare smiles in assay with smiles in correct_smiles:
        for cid in assay['PUBCHEM_CID']:
            if assay.loc[assay['PUBCHEM_CID'] == cid, 'PUBCHEM_EXT_DATASOURCE_SMILES'].values[0] != correct_isomeric_smiles.loc[correct_isomeric_smiles[0] == cid, 1].values[0]:
                non_isomeric_smi_cids[AID].append(cid)

        if len(non_isomeric_smi_cids[AID]) == 0:
            print(f'All SMILES in AID {AID} are isomeric')
        else:
            print(f'There are some non-isomeric SMILES in AID {AID}:')
            print(non_isomeric_smi_cids[AID])

    return non_isomeric_smi_cids

def update_isomeric(AIDs, non_isomeric_smi_cids):
    """
    Update the SMILES in the assay to isomeric SMILES.
    Input:
        AIDs (list of strings), non_isomeric_smi_cids (dictionary with AID as key and list of non-isomeric CIDs as values)
    """
    with open(f'{data_folder}/before_finished/step_2/non_isomeric_smi_cids.txt', 'w') as f:
        for AID in AIDs:
            assay = pd.read_csv(f'{data_folder}/before_finished/step_1/AID{AID}.csv')
            correct_isomeric_smiles = pd.read_csv(f'{data_folder}/before_finished/step_2/isomeric_smi_{AID}.txt', sep='\t', header=None)
            f.write(f'AID {AID}: {non_isomeric_smi_cids[AID]}\n')

            for cid in non_isomeric_smi_cids[AID]:
                f.write(f'CID {cid}: {assay.loc[assay["PUBCHEM_CID"] == cid, "PUBCHEM_EXT_DATASOURCE_SMILES"].values[0]} -> {correct_isomeric_smiles.loc[correct_isomeric_smiles[0] == cid, 1].values[0]}\n')
                assay.loc[assay['PUBCHEM_CID'] == cid, 'PUBCHEM_EXT_DATASOURCE_SMILES'] = correct_isomeric_smiles.loc[correct_isomeric_smiles[0] == cid, 1].values[0]

            assay.to_csv(f'{data_folder}/before_finished/step_2/AID{AID}.csv', index=False)

In [ ]:
non_isomeric_smi_cids = check_isomeric_smiles(AIDs)

All SMILES in AID 1488 are isomeric
There are some non-isomeric SMILES in AID 626:
[2997662, 2997957, 2999888]
All SMILES in AID 1741 are isomeric


Note: Here they returned that three smiles in AID626 were not isomeric. Again, this shows that the SMILES representation of some compounds in the given datasets might not be isomeric.

In [ ]:
update_isomeric(AIDs, non_isomeric_smi_cids)

# 3. InChI

We would like to include standard InChI to diversify users' choice of which data they would like to use for their own benchmark.

In [ ]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_3'):
    os.makedirs(f'{data_folder}/before_finished/step_3')

Again, it is convenient to use the PubChem Identifier Exchange Service (https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi) with operator type "same CID" and Output IDs "InChI" to retrieve InChI from a given list of input CIDs. The same CID lists from STEP 2 could be used here. The resulted InChIs could be checked if being standard by indentifying the presence of 'InChI=1S' at the begining of each InChI string.

In [ ]:
#Import AID626.csv as a dataframe:
AID626 = pd.read_csv(f'{data_folder}/before_finished/step_2/AID626.csv', sep=',', header=0)
AID1488 = pd.read_csv(f'{data_folder}/before_finished/step_2/AID1488.csv', sep=',', header=0)
AID1741 = pd.read_csv(f'{data_folder}/before_finished/step_2/AID1741.csv', sep=',', header=0)

In [ ]:
#import data
AID626_InChI = pd.read_csv(f'{data_folder}/before_finished/step_3/std_inchi_626.txt', sep='\t', header=None)

#Check if they are all standard InChI:
non_standard_InChI = []
for i in range(len(AID626_InChI[1])):
    if AID626_InChI[1][i].startswith('InChI=1S') == False:
        non_standard_InChI.append(AID626_InChI[1][i])
if non_standard_InChI == []:
    print('All InChI are standard')
else:
    print('There are some non-standard InChI')
    print(non_standard_InChI)

All InChI are standard


In [ ]:
#import data
AID1488_InChI = pd.read_csv(f'{data_folder}/before_finished/step_3/std_inchi_1488.txt', sep='\t', header=None)

#Check if they are all standard InChI:
non_standard_InChI = []
for i in range(len(AID1488_InChI[1])):
    if AID1488_InChI[1][i].startswith('InChI=1S') == False:
        non_standard_InChI.append(AID1488_InChI[1][i])
if non_standard_InChI == []:
    print('All InChI are standard')
else:
    print('There are some non-standard InChI')
    print(non_standard_InChI)

All InChI are standard


In [ ]:
#import data
AID1741_InChI = pd.read_csv(f'{data_folder}/before_finished/step_3/std_inchi_1741.txt', sep='\t', header=None)

#Check if they are all standard InChI:
non_standard_InChI = []
for i in range(len(AID1741_InChI[1])):
    if AID1741_InChI[1][i].startswith('InChI=1S') == False:
        non_standard_InChI.append(AID1741_InChI[1][i])
if non_standard_InChI == []:
    print('All InChI are standard')
else:
    print('There are some non-standard InChI')
    print(non_standard_InChI)

All InChI are standard


Now we concatenate the InChIs in our tables:

In [ ]:
#Convert AID626_InChI to a dictionary:
AID626_InChI_dict = dict(zip(AID626_InChI[0], AID626_InChI[1]))
AID1488_InChI_dict = dict(zip(AID1488_InChI[0], AID1488_InChI[1]))
AID1741_InChI_dict = dict(zip(AID1741_InChI[0], AID1741_InChI[1]))
#Add InChI to AID626 dataframe:
AID626['InChI'] = AID626['PUBCHEM_CID'].map(AID626_InChI_dict)
AID1488['InChI'] = AID1488['PUBCHEM_CID'].map(AID1488_InChI_dict)
AID1741['InChI'] = AID1741['PUBCHEM_CID'].map(AID1741_InChI_dict)

In [ ]:
#Convert all InChI to str:
AID626['InChI'] = AID626['InChI'].astype(str)
AID1488['InChI'] = AID1488['InChI'].astype(str)
AID1741['InChI'] = AID1741['InChI'].astype(str)

In [ ]:
#save:
AID626.to_csv(f'{data_folder}/before_finished/step_3/AID626.csv', index=False)
AID1488.to_csv(f'{data_folder}/before_finished/step_3/AID1488.csv', index=False)
AID1741.to_csv(f'{data_folder}/before_finished/step_3/AID1741.csv', index=False)

# 4. Check duplicates

When checking duplicates in the datasets, we would like to know if there are
1) Multiple identical molecules
2) Molecules with identical CID but different InChIs or SMILES
3) Molecules with identical InChI but with different CIDs or SMILES

In [ ]:
#import:
AID626 = pd.read_csv(f'{data_folder}/before_finished/step_3/AID626.csv', sep=',', header=0)
AID1488 = pd.read_csv(f'{data_folder}/before_finished/step_3/AID1488.csv', sep=',', header=0)
AID1741 = pd.read_csv(f'{data_folder}/before_finished/step_3/AID1741.csv', sep=',', header=0)

In [ ]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_4'):
    os.makedirs(f'{data_folder}/before_finished/step_4')

## 4.1. Checking identical molecules

In [ ]:
#Return all duplicates by comparing InChI:
AID626_duplicates_InChI = AID626[AID626.duplicated(subset=['InChI'], keep=False)]
AID1488_duplicates_InChI = AID1488[AID1488.duplicated(subset=['InChI'], keep=False)]
AID1741_duplicates_InChI = AID1741[AID1741.duplicated(subset=['InChI'], keep=False)]

print('Number of AID626 InChI duplicates: ', len(AID626_duplicates_InChI))
print('Number of AID1488 InChI duplicates: ', len(AID1488_duplicates_InChI))
print('Number of AID1741 InChI duplicates: ', len(AID1741_duplicates_InChI))

Number of AID626 InChI duplicates:  12
Number of AID1488 InChI duplicates:  0
Number of AID1741 InChI duplicates:  0


In [ ]:
#Return all duplicates by comparing SMILES:
AID626_duplicates_SMILES = AID626[AID626.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]
AID1488_duplicates_SMILES = AID1488[AID1488.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]
AID1741_duplicates_SMILES = AID1741[AID1741.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]

print('Number of AID626 SMILES duplicates: ', len(AID626_duplicates_SMILES))
print('Number of AID1488 SMILES duplicates: ', len(AID1488_duplicates_SMILES))
print('Number of AID1741 SMILES duplicates: ', len(AID1741_duplicates_SMILES))

Number of AID626 SMILES duplicates:  12
Number of AID1488 SMILES duplicates:  0
Number of AID1741 SMILES duplicates:  0


In [ ]:
#Return all duplicates by comparing CIDs:
AID626_duplicates_CIDs = AID626[AID626.duplicated(subset=['PUBCHEM_CID'], keep=False)]
AID1488_duplicates_CIDs = AID1488[AID1488.duplicated(subset=['PUBCHEM_CID'], keep=False)]
AID1741_duplicates_CIDs = AID1741[AID1741.duplicated(subset=['PUBCHEM_CID'], keep=False)]

print('Number of AID626 CID duplicates: ', len(AID626_duplicates_CIDs))
print('Number of AID1488 CID duplicates: ', len(AID1488_duplicates_CIDs))
print('Number of AID1741 CID duplicates: ', len(AID1741_duplicates_CIDs))

Number of AID626 CID duplicates:  12
Number of AID1488 CID duplicates:  0
Number of AID1741 CID duplicates:  0


In [ ]:
#write duplicates to a txt file:
with open(f'{data_folder}/before_finished/step_4/duplicates.txt', 'w') as f:
    f.write('AID626 InChI duplicates:\n')
    f.write(AID626_duplicates_InChI.to_string())
    f.write('\n\nAID1488 InChI duplicates:\n')
    f.write(AID1488_duplicates_InChI.to_string())
    f.write('\n\nAID1741 InChI duplicates:\n')
    f.write(AID1741_duplicates_InChI.to_string())
    f.write('\n\nAID626 SMILES duplicates:\n')
    f.write(AID626_duplicates_SMILES.to_string())
    f.write('\n\nAID1488 SMILES duplicates:\n')
    f.write(AID1488_duplicates_SMILES.to_string())
    f.write('\n\nAID1741 SMILES duplicates:\n')
    f.write(AID1741_duplicates_SMILES.to_string())
    f.write('\n\nAID626 CID duplicates:\n')
    f.write(AID626_duplicates_CIDs.to_string())
    f.write('\n\nAID1488 CID duplicates:\n')
    f.write(AID1488_duplicates_CIDs.to_string())
    f.write('\n\nAID1741 CID duplicates:\n')
    f.write(AID1741_duplicates_CIDs.to_string())

## 4.2. Same CIDs but different chemical representations

In [ ]:
#reindex
AID626_duplicates_CIDs.reset_index(drop=True, inplace=True)

In [ ]:
sameCID_differentInChI = []

for i in range(len(AID626_duplicates_CIDs['PUBCHEM_CID'])):
    for j in range(i+1, len(AID626_duplicates_CIDs['PUBCHEM_CID'])):
        if AID626_duplicates_CIDs['PUBCHEM_CID'][i] == AID626_duplicates_CIDs['PUBCHEM_CID'][j]:
            if AID626_duplicates_CIDs['InChI'][i] != AID626_duplicates_CIDs['InChI'][j]:
                sameCID_differentInChI.append((AID626_duplicates_CIDs['PUBCHEM_CID'][i], AID626_duplicates_CIDs['PUBCHEM_CID'][j]))

if sameCID_differentInChI == []:
    print('No duplicate CIDs with different InChIs')
else:
    print('Duplicate CIDs with different InChIs')
    print(sameCID_differentInChI)

No duplicate CIDs with different InChIs


In [ ]:
sameCID_differentSMILES = []

for i in range(len(AID626_duplicates_CIDs['PUBCHEM_CID'])):
    for j in range(i+1, len(AID626_duplicates_CIDs['PUBCHEM_CID'])):
        if AID626_duplicates_CIDs['PUBCHEM_CID'][i] == AID626_duplicates_CIDs['PUBCHEM_CID'][j]:
            if AID626_duplicates_CIDs['PUBCHEM_EXT_DATASOURCE_SMILES'][i] != AID626_duplicates_CIDs['PUBCHEM_EXT_DATASOURCE_SMILES'][j]:
                sameCID_differentSMILES.append((AID626_duplicates_CIDs['PUBCHEM_CID'][i], AID626_duplicates_CIDs['PUBCHEM_CID'][j]))

if sameCID_differentSMILES == []:
    print('No duplicate CIDs with different SMILES')
else:
    print('Duplicate CIDs with different SMILES')
    print(sameCID_differentSMILES)

No duplicate CIDs with different SMILES


## 4.3. Same InChI but with different CIDs or SMILES

In [ ]:
AID626_duplicates_InChI.reset_index(drop=True, inplace=True)

In [ ]:
sameInChI_differentCID = []

for i in range(len(AID626_duplicates_InChI['InChI'])):
    for j in range(i+1, len(AID626_duplicates_InChI['InChI'])):
        if AID626_duplicates_InChI['InChI'][i] == AID626_duplicates_InChI['InChI'][j]:
            if AID626_duplicates_InChI['PUBCHEM_CID'][i] != AID626_duplicates_InChI['PUBCHEM_CID'][j]:
                sameInChI_differentCID.append((AID626_duplicates_InChI['PUBCHEM_CID'][i], AID626_duplicates_InChI['PUBCHEM_CID'][j]))

if sameInChI_differentCID == []:
    print('No duplicate InChIs with different CIDs')
else:
    print('Duplicate InChIs with different CIDs')
    print(sameInChI_differentCID)

No duplicate InChIs with different CIDs


In [ ]:
sameInChI_differentSMILES = []

for i in range(len(AID626_duplicates_InChI['InChI'])):
    for j in range(i+1, len(AID626_duplicates_InChI['InChI'])):
        if AID626_duplicates_InChI['InChI'][i] == AID626_duplicates_InChI['InChI'][j]:
            if AID626_duplicates_InChI['PUBCHEM_EXT_DATASOURCE_SMILES'][i] != AID626_duplicates_InChI['PUBCHEM_EXT_DATASOURCE_SMILES'][j]:
                sameInChI_differentSMILES.append((AID626_duplicates_InChI['PUBCHEM_CID'][i], AID626_duplicates_InChI['PUBCHEM_CID'][j]))

if sameInChI_differentSMILES == []:
    print('No duplicate InChIs with different SMILES')
else:
    print('Duplicate InChIs with different SMILES')
    print(sameInChI_differentSMILES)

No duplicate InChIs with different SMILES


## 4.4 Drop duplicates

When dropping duplicates, we will keep the first molecule in a pair or a group of duplicates. For example, here there are 12 duplicates (6 pairs) so we keep 6 of them.

In [ ]:
# Keep only the first duplicate in the dataframes:
AID626.drop_duplicates(subset=['InChI'], keep='first', inplace=True)
AID1488.drop_duplicates(subset=['InChI'], keep='first', inplace=True)
AID1741.drop_duplicates(subset=['InChI'], keep='first', inplace=True)


In [ ]:
if len(AID626[AID626.duplicated(subset=['InChI'], keep=False)]) == 0:
    print('No more duplicate InChI in AID626')
else:
    print('There are still duplicate InChI in AID626')

No more duplicate InChI in AID626


In [ ]:
# Save the dataframes to csv:
AID626.to_csv(f'{data_folder}/before_finished/step_4/AID626.csv', index=False)
AID1488.to_csv(f'{data_folder}/before_finished/step_4/AID1488.csv', index=False)
AID1741.to_csv(f'{data_folder}/before_finished/step_4/AID1741.csv', index=False)

# 5. Hierarchical Curation

For the hierarchical curation, there are some rules:

(1) All assays used should be on the same or close species/cell lines. Optimally, they should also be from the same project/laboratory.

(2) Primary actives (PrA) will have a large false-positive rate. Therefore, they should be tested in follow-up confirmatory screens (optimally dose-reponse).

(3) Actives could be promiscuous. Therefore, it is optimal to have counter-screens on different targets to test specificity.

(4) For some projects, compounds were tested in multiple rounds. Therefore, assays often have hierarchical relations. From a single primary screen (Pr), active compounds (Pr_A) could be tested in multiple rounds of confirmatory screens (Cf_1, Cf_2, ..., Cf_final) or counter screens (Ct_1, Ct_2, etc.). Actives from confirmatory screens (Cf_A) have a higher possibility of being true active. If an active compound is tested active in counter screens (Ct_A), it is likely to be a promiscuous compound and should not be included.

(4) It is important to know the relationship between assays. Active sets from downstream screens always have a lower false-positive rate than active sets from upstream screens due to better assay technologies on a smaller set of compounds. Therefore, final hits should be taken from the intersection of the very last confirmatory assays, without tested active in any counter-screens:
Final hits = [Cf_final1_A ∩ Cf_final2_A ∩ ...] \ [Ct_1_A ∪ Ct_2_A ∪ ...]

However, if the confirmatory assays are unrelated (tested on different set of compounds), then we might have to take the union of their active sets instead of the intersections as in this formula.

(5) The hierarchical relations should be inspected carefully to see if follow-up confirmatory screens include extra compounds (Ex) that were not tested in earlier screens or tested inactive in earlier screens. If exist, these compounds require manual inspection.

(6) Final inactives should be taken from primary inactives (Pr_I) (not inconclusive, unspecified, or probes), plus extra compounds that were tested inactive in conformatory screens (Ex_I), if justified.
Final inactives = PrI ∪ Ex_I



## 5.1. Classify groups of compounds in each assay by activities

In [ ]:
path = f'{data_folder}/before_finished/step_4'
keynumbers = [626, 1488, 1741] # specify the keynumbers you want to import

for keynumber in keynumbers:
    filename = os.path.join(path, f'AID{keynumber}.csv')
    if os.path.exists(filename):
        df = pd.read_csv(filename, index_col=None, header=0)
        exec(f'AID{keynumber} = df')
        exec(f'AID{keynumber}_active = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Active"]')
        exec(f'AID{keynumber}_inactive = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Inactive"]')
        exec(f'AID{keynumber}_inconclusive = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Inconclusive"]')
        exec(f'AID{keynumber}_unspecified = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Unspecified"]')
        exec(f'AID{keynumber}_probe = df[df["PUBCHEM_ACTIVITY_OUTCOME"]=="Probe"]')

In [ ]:
#Create a df with first column the variables name, and the second column the number of rows:
df = pd.DataFrame(columns=['AID', 'Tested Compounds', 'Active', 'Inactive', 'Inconclusive', 'Unspecified', 'Probe'])
for keynumber in keynumbers:
    exec(f'df.loc[len(df)] = ["AID{keynumber}", len(AID{keynumber}), len(AID{keynumber}_active), len(AID{keynumber}_inactive), len(AID{keynumber}_inconclusive), len(AID{keynumber}_unspecified), len(AID{keynumber}_probe)]')
df

,AID,Tested Compounds,Active,Inactive,Inconclusive,Unspecified,Probe
0,AID626,63676,1938,61738,0,0,0
1,AID1488,1665,309,940,416,0,0
2,AID1741,1665,150,1195,320,0,0


## 5.2. Check the hierachical relations

In [ ]:
# Check if compounds tested in coonformatory AID1488 was active in the primary screen 626
AID1488['PUBCHEM_CID'].isin(AID626_active['PUBCHEM_CID']).value_counts()

PUBCHEM_CID
True    1665
Name: count, dtype: int64

In [ ]:
# Check if compounds tested in coonformatory AID1488 was active in the primary screen 626
AID1741['PUBCHEM_CID'].isin(AID626_active['PUBCHEM_CID']).value_counts()

PUBCHEM_CID
True    1665
Name: count, dtype: int64

In [ ]:
# Check how many active compounds in AID1488 were tested active in AID1741.
AID1488_active['PUBCHEM_CID'].isin(AID1741_active['PUBCHEM_CID']).value_counts()

PUBCHEM_CID
False    188
True     121
Name: count, dtype: int64

In [ ]:
len(AID1741_active) - AID1488_active['PUBCHEM_CID'].isin(AID1741_active['PUBCHEM_CID']).value_counts()

PUBCHEM_CID
False   -38
True     29
Name: count, dtype: int64

There is no problem with this assay data. So now we take actives in AID 1488 that were inactive in AID1741 as potential hits, which renders 188 compounds. And we take all inactives in AID626 as potential inactives.

## 5.3. Export the data

In [ ]:
#Subtract the CIDs from AID1741_active from AID1488_active:
potential_hits_cids = AID1488_active[~AID1488_active['PUBCHEM_CID'].isin(AID1741_active['PUBCHEM_CID'])]

#Get all the rows from AID626 that are in final_hits:
potential_hits_M1_agonist = AID626[AID626['PUBCHEM_CID'].isin(potential_hits_cids['PUBCHEM_CID'])]

In [ ]:
#Get all the rows from AID626 that are inactive
potential_inactives_M1_agonist = AID626_inactive

In [ ]:
if not os.path.exists(f'{data_folder}/before_finished/step_5'):
    os.makedirs(f'{data_folder}/before_finished/step_5')

In [ ]:
#export the potential hits and inactives to csv:
potential_hits_M1_agonist.to_csv(f'{data_folder}/before_finished/step_5/potential_hits_M1_agonist.csv', index=False)
potential_inactives_M1_agonist.to_csv(f'{data_folder}/before_finished/step_5/potential_inactives_M1_agonist.csv', index=False)

# 6. RDkit check

In [ ]:
potential_hits_M1_agonist = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_hits_M1_agonist.csv', sep=',', header=0)
potential_inactives_M1_agonist = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_inactives_M1_agonist.csv', sep=',', header=0)

In [ ]:
def process_smiles(df):
    """
    This function check if the SMILES strings from a given dataset could be parsed by RDKit or if they returns any problems detected by RDkit
    Input:
        Pandas dataframe
    Output:
        mol_list: dictionary with CID as key and RDKit molecule object as value
        problem_list: list of problems detected by RDKit
        cannot_parse: list of CIDs that could not be parsed by RDKit
    """
    mol_list = {}
    problem_list = []
    cannot_parse = []

    for i in df['PUBCHEM_CID']:

        #convert each SMILES to molecule:
        m = Chem.MolFromSmiles(df[df['PUBCHEM_CID'] == i]['PUBCHEM_EXT_DATASOURCE_SMILES'].values[0], sanitize=False)
        mol_list[i] = m

        if m is None:
            cannot_parse.append(i) #save if molecule is non-parsable

        elif m is not None:
            problems = Chem.DetectChemistryProblems(m) #identify and capture error messages when creating mol objects.
        if problems != ():
            problem_list.append(problems)

    if len(problem_list) > 0:
        print(problem_list)
    else:
        print("No problems detected")
    return mol_list, problem_list, cannot_parse

In [ ]:
mol_hits, problem_list_hits, cannot_parse_hits = process_smiles(potential_hits_M1_agonist)
mol_inactives, problem_list_inactives, cannot_parse_inactives = process_smiles(potential_inactives_M1_agonist)

No problems detected
No problems detected


In [ ]:
#Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_6'):
    os.makedirs(f'{data_folder}/before_finished/step_6')

#if there is problem in the problem list or molecule that is non-parsable, save these data into step_6 folder.

Our dataset returned no problem or non-parsable molecule.

# 7. Inorganics

In [ ]:
# Import data
potential_hits_M1_agonist = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_hits_M1_agonist.csv', sep=',', header=0)
potential_inactives_M1_agonist = pd.read_csv(f'{data_folder}/before_finished/step_5/potential_inactives_M1_agonist.csv', sep=',', header=0)

In [ ]:
# Convert to mol objects
mol_list_hits = [Chem.MolFromSmiles(smi, sanitize=True) for smi in potential_hits_M1_agonist['PUBCHEM_EXT_DATASOURCE_SMILES']]
mol_list_inactives = [Chem.MolFromSmiles(smi, sanitize=True) for smi in potential_inactives_M1_agonist['PUBCHEM_EXT_DATASOURCE_SMILES']]

In [ ]:
#Filter out inorganic molecules using thermo's functional_groups module
organic_mols_hits = [mol for mol in mol_list_hits if mol is not None and not functional_groups.is_inorganic(mol)]
inorganic_mols_hits = [mol for mol in mol_list_hits if mol is not None and functional_groups.is_inorganic(mol) == True]
organic_mols_inactives = [mol for mol in mol_list_inactives if mol is not None and not functional_groups.is_inorganic(mol)]
inorganic_mols_inactives = [mol for mol in mol_list_inactives if mol is not None and functional_groups.is_inorganic(mol) == True]


print(f'In hits, there are {len(organic_mols_hits)} organic molecules and {len(inorganic_mols_hits)} inorganic molecules')
print(f'In inactives, there are {len(organic_mols_inactives)} organic molecules and {len(inorganic_mols_inactives)} inorganic molecules')

In hits, there are 188 organic molecules and 0 inorganic molecules
In decoys, there are 61738 organic molecules and 0 inorganic molecules


# 8. Mixture

## 8.1. Quick check

In [ ]:
def quick_check_mixtures(name, smiles_list):
    count=0
    for smiles in smiles_list:
        if '.' in smiles:
            count+=1
    print(f"Total number of mixtures in {name} is {count}")

quick_check_mixtures('hits', potential_hits_M1_agonist['PUBCHEM_EXT_DATASOURCE_SMILES'])
quick_check_mixtures('inactives', potential_inactives_M1_agonist['PUBCHEM_EXT_DATASOURCE_SMILES'])

Total number of mixtures in hits is 13
Total number of mixtures in decoys is 1906


## 8.2. Handling mixture

In [ ]:
def process_smiles_dataframe(df):
    """
    From a given dataframe, detect and handle mixtures based on SMILES representation.
    Input:
        Pandas dataframe
    Output:
        (dictionary: CID -> SMILES)
            processed: non-mixture forms of every SMILES in the given dataset
            removed: mixture components or mixtures removed from the original mixture SMILES
            small_organic: small organic molecules that are removed from a size-imbalanced mixtures of organic molecules
            small_inorganic: small inorganic molecules that are removed from a mixture of both organic and inorganic molecules
            big_organic_not_lipinski: large organic molecules that are not kept due to not passing the lipinski criteria
            cleaned_from_mixtures: non-mixture forms after handling of the orignal mixtures
    """
    processed = {}
    removed = {}
    small_inorganic = {}
    small_organic = {}
    big_organic_not_lipinski = {}
    cleaned_from_mixtures = {}

    for index, row in df.iterrows():
        cid = row['PUBCHEM_CID']
        smiles = row['PUBCHEM_EXT_DATASOURCE_SMILES']

        if '.' in smiles: # Check for mixtures
            molecules = smiles.split('.')
            mols = [Chem.MolFromSmiles(mol) for mol in molecules]
            num_atoms = [mol.GetNumAtoms() for mol in mols if mol is not None]

            if all(x == num_atoms[0] for x in num_atoms): # Check if all molecules have the same number of atoms. If yes, keep one of them
                processed[cid] = molecules[0]
                cleaned_from_mixtures[cid] = molecules[0]
                removed[cid] = '.'.join(molecules[1:])
            else:
                if max(num_atoms) - min(num_atoms) <= 5:
                    removed[cid] = smiles # Remove the mixture if the difference in number of atoms is less than or equal to 5
                    print(f"Cannot decide between {molecules} for CID {cid}")
                else:
                    max_index = num_atoms.index(max(num_atoms))
                    min_index = num_atoms.index(min(num_atoms))
                    if functional_groups.is_inorganic(mols[min_index]) == True:
                        processed[cid] = molecules[max_index]
                        cleaned_from_mixtures[cid] = molecules[max_index]
                        removed[cid] = molecules[min_index] # Keep the organic molecule and remove the inorganic one
                        small_inorganic[cid] = molecules[min_index]
                    else:
                        big_molecule = mols[max_index]

                        # Calculate properties for Lipinski's rule of five
                        mw = Descriptors.MolWt(big_molecule)
                        hbd = rdMolDescriptors.CalcNumHBD(big_molecule)
                        hba = rdMolDescriptors.CalcNumHBA(big_molecule)
                        logp = Crippen.MolLogP(big_molecule)

                        # Check Lipinski's criteria
                        if mw <= 500 and hbd <= 5 and hba <= 10 and logp <= 5:
                            processed[cid] = molecules[max_index] # Keep the big organic molecule if it passes Lipinski's rule of five
                            cleaned_from_mixtures[cid] = molecules[max_index]
                            removed[cid] = '.'.join([molecules[i] for i in range(len(molecules)) if i != max_index])
                            small_organic[cid] = molecules[min_index]
                        else:
                            removed[cid] = smiles # Remove the mixture if the big organic molecule does not pass Lipinski's rule of five
                            big_organic_not_lipinski[cid] = molecules[max_index]
                            print(f"Big organic molecule for CID {cid} does not pass Lipinski's rule of five")

        else:
            processed[cid] = smiles

    return processed, removed, small_organic, small_inorganic, big_organic_not_lipinski, cleaned_from_mixtures


In [ ]:
processed_hits, removed_hits, small_organic_hits, small_inorganic_hits, not_lipinski_hits, cleaned_hits = process_smiles_dataframe(potential_hits_M1_agonist)
processed_inactives, removed_inactives, small_organic_inactives, small_inorganic_inactives, not_lipinski_inactives, cleaned_inactives = process_smiles_dataframe(potential_inactives_M1_agonist)

Cannot decide between ['CC1=NC2=C(O1)C3=CC=CC=C3C=C2', 'C1=C(C=C(C(=C1[N+](=O)[O-])O)[N+](=O)[O-])[N+](=O)[O-]'] for CID 3241713
Big organic molecule for CID 3244813 does not pass Lipinski's rule of five
Cannot decide between ['CN(C)C1=NC2=CC=CC=C2C(=C1)N', 'C1=C(NC(=O)NC1=O)C(=O)O'] for CID 646688
Cannot decide between ['C1C2(CN3CN1CN(C2)C3)N', 'C1=CC(=CN=C1)C(=O)O'] for CID 648270
Cannot decide between ['CN(C)C1=NC2=C(CCC2)C(=C1)N', 'C1=C(C=NC=C1O)C(=O)O'] for CID 652177
Cannot decide between ['CC1=NC2=C(S1)C3=CC=CC=C3C=C2', 'C1=C(C=C(C(=C1[N+](=O)[O-])O)[N+](=O)[O-])[N+](=O)[O-]'] for CID 654127
Cannot decide between ['C1CC1C(C2CC2)NC3=NCCO3', 'C(=C/C(=O)O)\\C(=O)O'] for CID 5388964
Cannot decide between ['CN1C(=O)C2=C(N=C(N2)Cl)N(C1=O)C(=O)[O-]', 'CN(C)CCOC(C1=CC=CC=C1)C2=CC=CC=C2'] for CID 657227
Cannot decide between ['CC[N+](C)(C)CC1=CC=CC=C1Br', 'CC1=CC=C(C=C1)S(=O)(=O)[O-]'] for CID 6100
Cannot decide between ['C1CN(CCN1CCOCCO)C(C2=CC=CC=C2)C3=CC=C(C=C3)Cl', 'C1=CC=C2C(=C1)C=C

Before updating the dataset, need to update the InChI for the mixtures.

In [ ]:
# Create a new step folder
if not os.path.exists(f'{data_folder}/before_finished/step_8'):
    os.makedirs(f'{data_folder}/before_finished/step_8')

In [ ]:
#Generate df with the smiles column in the cleaned_hits or cleaned_inactives dictionary:
cleaned_hits_df = pd.DataFrame(list(cleaned_hits.values()), columns=['PUBCHEM_EXT_DATASOURCE_SMILES'])
cleaned_inactives_df = pd.DataFrame(list(cleaned_inactives.values()), columns=['PUBCHEM_EXT_DATASOURCE_SMILES'])

In [ ]:
#Export the cleaned hits and inactives to csv:
cleaned_hits_df.to_csv(f'{data_folder}/before_finished/step_8/cleaned_mixtures_hits_M1_agonist.csv', index=False, header=False)
cleaned_inactives_df.to_csv(f'{data_folder}/before_finished/step_8/cleaned_mixtures_inactives_M1_agonist.csv', index=False, header=False)

In [ ]:
def process_mixture_df(name, df, processed, removed, small_organic, small_inorganic):
    """
    Update a given dataframe with information on mixture handling
    """
    indices_to_drop = []  # List to keep track of row indices that should be dropped

    for index, row in df.iterrows():
        cid = row['PUBCHEM_CID']
        if cid in processed:
            df.loc[index, 'PUBCHEM_EXT_DATASOURCE_SMILES'] = processed[cid]
            if cid in removed:
                df.loc[index, 'Mol removed from mixture'] = removed[cid]
            if cid in small_organic:
                df.loc[index, 'Small organic molecule'] = small_organic[cid]
            if cid in small_inorganic:
                df.loc[index, 'Small inorganic molecule'] = small_inorganic[cid]
        else:
            indices_to_drop.append(index)
            print(f"{cid} has been removed from {name} because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.")

    # Drop rows outside the loop and reset index if needed
    new_df = df.drop(indices_to_drop).reset_index(drop=True)

    return new_df

processed_hits_df = process_mixture_df('hits_M1_agonist', potential_hits_M1_agonist, processed_hits, removed_hits, small_organic_hits, small_inorganic_hits)
processed_inactives_df = process_mixture_df('inactives_M1_agonist', potential_inactives_M1_agonist, processed_inactives, removed_inactives, small_organic_inactives, small_inorganic_inactives)

3241713 has been removed from decoys_M1_agonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
3244813 has been removed from decoys_M1_agonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
646688 has been removed from decoys_M1_agonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
648270 has been removed from decoys_M1_agonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
652177 has been removed from decoys_M1_agonist because it is a mixture with less than 5 atoms difference or the big organic molecule does not pass Lipinski's rule of five.
654127 has been removed from decoys_M1_agonist because it is a mixture with less than 5 atoms difference or the big organic molecule does 

In [ ]:
print(
    f'Hits before processing: {len(potential_hits_M1_agonist)}\n'
    f'Hits after processing: {len(processed_hits_df)}\n'
    f'Mixtures detected: {len(removed_hits)}\n'
    f'Mixtures with small inorganic molecules: {len(small_inorganic_hits)}\n'
    f'Mixtures with big organic molecules passing Lipinski: {len(small_organic_hits)}\n'
    f'Mixtures with big organic molecules not passing Lipinski: {len(not_lipinski_hits)}\n'
)

print(
    f'Inactives before processing: {len(potential_inactives_M1_agonist)}\n'
    f'Inactives after processing: {len(processed_inactives_df)}\n'
    f'Mixtures detected: {len(removed_inactives)}\n'
    f'Mixtures with small inorganic molecules: {len(small_inorganic_inactives)}\n'
    f'Mixtures with big organic molecules passing Lipinski: {len(small_organic_inactives)}\n'
    f'Mixtures with big organic molecules not passing Lipinski: {len(not_lipinski_inactives)}\n'
)

# Save the processed dataframes to csv
processed_hits_df.to_csv(f'{data_folder}/before_finished/step_8/post8_hits.csv', index=False)
processed_inactives_df.to_csv(f'{data_folder}/before_finished/step_8/post8_inactives.csv', index=False)

print('Dataframes saved successfully')

Hits before processing: 188
Hits after processing: 188
Mixtures detected: 13
Mixtures with small inorganic molecules: 13
Mixtures with big organic molecules passing Lipinski: 0
Mixtures with big organic molecules not passing Lipinski: 0

Decoys before processing: 61738
Decoys after processing: 61725
Mixtures detected: 1906
Mixtures with small inorganic molecules: 1826
Mixtures with big organic molecules passing Lipinski: 66
Mixtures with big organic molecules not passing Lipinski: 4

Dataframes saved successfully


# 9. Neutralize & 10. Aromatize molecules

In [ ]:
# Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_9_10'):
    os.makedirs(f'{data_folder}/before_finished/step_9_10')

In [ ]:
#import:
pre9_hits = pd.read_csv(f'{data_folder}/before_finished/step_8/post8_hits.csv', sep=',', header=0)
pre9_inactives = pd.read_csv(f'{data_folder}/before_finished/step_8/post8_inactives.csv', sep=',', header=0)

In [ ]:
def neutralize_atoms(mol):
    """
    Code adapted from https://www.rdkit.org/docs/Cookbook.html.
    Source: https://baoilleach.blogspot.com/2019/12/no-charge-simple-approach-to.html
    (Noel O’Boyle, 2019)

    This function return a neutralized molecules for a given input Mol object.
    Additional handling was added for molecules with tetracoordinated boron.
    """
    pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
    at_matches = mol.GetSubstructMatches(pattern)
    at_matches_list = [y[0] for y in at_matches]
    if len(at_matches_list) > 0:
        for at_idx in at_matches_list:
            atom = mol.GetAtomWithIdx(at_idx)
            chg = atom.GetFormalCharge()
            hcount = atom.GetTotalNumHs()

            #Skip adjustment for tetracoordinated boron
            if atom.GetAtomicNum() == 5 and atom.GetDegree() == 4: #ADD COMMENT
                continue  # Just bypass the problematic atom

            atom.SetFormalCharge(0)
            atom.SetNumExplicitHs(hcount - chg)
            atom.UpdatePropertyCache()
    return mol

In [ ]:
def aromatize_smile(mol):
    """
    This function dekekulize an input Mol object and return the aromatic form of isomeric SMILES.
    """
    Chem.Kekulize(mol)
    Chem.SanitizeMol(mol, Chem.SanitizeFlags.SANITIZE_ALL)
    aromatic_smiles = Chem.MolToSmiles(mol, isomericSmiles = True)
    return aromatic_smiles

In [ ]:
updated_smi = []

#Update dataset with neutralized, aromatic SMILES
for smi in pre9_hits['PUBCHEM_EXT_DATASOURCE_SMILES']:
    mol = Chem.MolFromSmiles(smi)
    mol_neu = neutralize_atoms(mol)
    smi_arom = aromatize_smile(mol_neu)
    updated_smi.append(smi_arom)

#update the smiles in this df
pre9_hits['PUBCHEM_EXT_DATASOURCE_SMILES'] = updated_smi


In [ ]:
updated_smi = []

#Update dataset with neutralized, aromatic SMILES
for smi in pre9_inactives['PUBCHEM_EXT_DATASOURCE_SMILES']:
    mol = Chem.MolFromSmiles(smi)
    mol_neu = neutralize_atoms(mol)
    smi_arom = aromatize_smile(mol_neu)
    updated_smi.append(smi_arom)

#update the smiles in this df
pre9_inactives['PUBCHEM_EXT_DATASOURCE_SMILES'] = updated_smi


In [ ]:
#Save
pre9_hits.to_csv(f'{data_folder}/before_finished/step_9_10/post10_hits.csv', index=False)
pre9_inactives.to_csv(f'{data_folder}/before_finished/step_9_10/post10_inactives.csv', index=False)

# Post 9+10: Update InChI


Is it important to now update InChI in our datasets, for 2 reasons:

(1) Some mixture compounds have been modified (removal of small inorganic or organic molecules) in SMILES representation but not InChIs.

(2) The SMILES representations have been neutralized and aromatized, but not InChIs.

In [ ]:
#export the smiles columns to txt
pre9_hits['PUBCHEM_EXT_DATASOURCE_SMILES'].to_csv(f'{data_folder}/before_finished/step_9_10/smiles_hits.txt', index=False, header=False)
pre9_inactives['PUBCHEM_EXT_DATASOURCE_SMILES'].to_csv(f'{data_folder}/before_finished/step_9_10/smiles_inactives.txt', index=False, header=False)

In [ ]:
#Import the converted InChIs
cleaned_inchi_hits = pd.read_csv(f'{data_folder}/before_finished/step_9_10/inchi_hits.txt', sep='\t', header=None)
cleaned_inchi_inactives = pd.read_csv(f'{data_folder}/before_finished/step_9_10/inchi_inactives.txt', sep='\t', header=None)

#a dictionary of smiles and corresponding inchi in cleaned_inchi_hits
hits_smi_inchi_dict = dict(zip(cleaned_inchi_hits[0], cleaned_inchi_hits[1]))
inactives_smi_inchi_dict = dict(zip(cleaned_inchi_inactives[0], cleaned_inchi_inactives[1]))

#update the pre9_hits by matching the smiles with keys and replace inchi with values:
pre9_hits['InChI'] = pre9_hits['PUBCHEM_EXT_DATASOURCE_SMILES'].map(hits_smi_inchi_dict)
pre9_inactives['InChI'] = pre9_inactives['PUBCHEM_EXT_DATASOURCE_SMILES'].map(inactives_smi_inchi_dict)


In [ ]:
#export:
pre9_hits.to_csv(f'{data_folder}/before_finished/step_9_10/post10_hits.csv', index=False)
pre9_inactives.to_csv(f'{data_folder}/before_finished/step_9_10/post10_inactives.csv', index=False)

# 11. PAIN filters

## 11.1. Frequency of hits (FoH)

Frequency of Hits is a complex concept that requires a merticulous approach. In general, the rule is if a compound was tested active in multiple assays, it is likely to be a promiscuous compound.
1. For each compounds, retrieve the information on its tested assays
2. For each of the assay tested, retrieve the sequence of the protein target.
3. Given all sequence of the protein tested, do a multiple sequence alignment to find the percentage Percent Identity (similarty) between these proteins. If an assay has high percentage to other targets, then these assays contribute less to promiscuousity of the compound.
4. Use the percentage identity as a weight:
w = 1 - %SI/100
Calculate the frequency of hits for each compound:
FoH = wACC/TAC
wACC is the weighed total number of assay tested where the compounds were identified acitives. TAC is the total number of assays tested.

In [ ]:
pre11_hits = pd.read_csv(f'{data_folder}/before_finished/step_9_10/post10_hits.csv', sep=',', header=0)
pre11_inactives = pd.read_csv(f'{data_folder}/before_finished/step_9_10/post10_inactives.csv', sep=',', header=0)

In [ ]:
# Create a new step folder:
if not os.path.exists(f'{data_folder}/before_finished/step_11/11_1'):
    os.makedirs(f'{data_folder}/before_finished/step_11/11_1')

### 11.1.1. PubChem testing information for each compound

This part illustrates how to retrieve the information of how each compound was tested from the PubChem database. Bulk data retrieval from the ftp server is used to get the information of every bioassay in PubChem:

In [ ]:
url = 'https://ftp.ncbi.nlm.nih.gov/pubchem/Bioassay/Extras/bioassays.tsv.gz' #this FTP file records the summary data of all available AIDs in PubChem

local_save_dir = 'H:\coding\HiChem\curation\pubchem_sum'
local_save_path = os.path.join(local_save_dir, 'bioassays.tsv.gz')

if not os.path.exists(local_save_dir):
    os.makedirs(local_save_dir)
r = requests.get(url, stream=True)

with open(local_save_path, 'wb') as f:
    for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)
print('Downloaded to %s' % local_save_path)

In [ ]:
path = 'H:/coding/HiChem/curation/pubchem_sum/bioassays.tsv.gz'

# Read the TSV file
all_bioassay = pd.read_csv(path, delimiter='\t')

In [ ]:
all_bioassay.head()

,AID,BioAssay Name,Deposit Date,Modify Date,Source Name,Source ID,Substance Type,Outcome Type,Project Category,BioAssay Group,BioAssay Types,Protein Accessions,UniProts IDs,Gene IDs,Target TaxIDs,Taxonomy IDs,Number of Tested SIDs,Number of Active SIDs,Number of Tested CIDs,Number of Active CIDs
0,1,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,55004,3280,52994,3057
1,3,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,51204,2596,49337,2449
2,5,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,53832,2486,51804,2301
3,7,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,53828,4275,51804,4041
4,9,NCI human tumor cell line growth inhibition as...,20040815,20231014,DTP/NCI,NCI human tumor cell line growth inhibition as...,small-molecule,Confirmatory,Other,NCI-60_DOSERESP,NaN,NaN,NaN,NaN,NaN,NaN,53744,3125,51771,2948


### 11.1.2 Retrieving protein sequences for assays tested:

Then, the testing information for each compound is retrieved from the PugREST API

In [ ]:
# Cache to store the number of compounds tested per AID to avoid redundant call.
num_compounds_tested_cache = {}

def get_num_compounds_tested(aid, all_bioassay=all_bioassay):
    """
    This function retrieves the information of how many compounds were tested in a given assay (by AID).
    """
    if aid in num_compounds_tested_cache:
        return num_compounds_tested_cache[aid]
    else:
        #return the 'Number of Tested CIDs' column value at the row where the 'AID' column is equal to aid in the all_bioassay dataframe
        num_compounds_tested = all_bioassay[all_bioassay['AID'] == aid]['Number of Tested CIDs'].values[0]
    return num_compounds_tested

def get_assay_data(cid):
    """
    Return a dictionary of all targets that a given compound (by CID) was tested on in PubChem
    and the activity values of the compound.
    """
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/assaysummary/JSON" #PUG-REST compound summary by CID
    response = requests.get(url)
    data = response.json()

    target_activity = {}

    if 'Table' in data and 'Row' in data['Table']:
        for row in data['Table']['Row']:
            cells = row['Cell']
            aid = int(cells[0])  # Extracting the AID from the first cell

            # Proceed only if the assay is a screening assay
            if cells[10] == 'Screening':

                # Proceed only if more than 10,000 compounds were tested
                num_compounds_tested = get_num_compounds_tested(aid)
                if num_compounds_tested > 10000:
                    target_gi = cells[5] # Retrieve the protein target's GI
                    activity_outcome = cells[4].lower()

                    if target_gi not in target_activity:
                        target_activity[target_gi] = activity_outcome == 'active'
                    elif activity_outcome == 'active':
                        target_activity[target_gi] = True # If a compound was tested multiple times on the same protein, priotize "active" outcome.

            else:
                continue

    return (cid, target_activity)

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

cids_list = pre11_hits['PUBCHEM_CID'].tolist()

def execute_with_multiprocessing(cids_list):
    """
    For a given list of CIDs, return a dictionary of dictionaries
    of protein targets these compounds were tested on and the activity outcomes
    Input:
        [list of CIDs]
    Output:
        Dictionary of testing information for all CIDs, such as:
        {CID1:{target1:activity1, target3:activity3, ...},{CID2:{target2:activity2, target4:activity4, ...}, ...}}
    """
    results_dict = {}
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Prepare futures for all CIDs
        futures = [executor.submit(get_assay_data, cid) for cid in cids_list]

        # Process futures as they complete
        for future in tqdm(as_completed(futures), total=len(cids_list), desc="Processing CIDs"):
            try:
                cid, target_activity = future.result()
                results_dict[cid] = target_activity
            except Exception as e:
                print(f"Error processing CID: {e}")
    return results_dict

results_dict = execute_with_multiprocessing(cids_list)

Processing CIDs:   0%|          | 0/188 [00:00<?, ?it/s]

Processing CIDs: 100%|██████████| 188/188 [01:20<00:00,  2.32it/s]


In [ ]:
#export results_dict
with open(f'{data_folder}/before_finished/step_11/11_1/results_dict.json', 'w') as f:
    json.dump(results_dict, f)

In [ ]:
#get the list of all keys of the values in the dictionary:
protein_ids = []
for value in results_dict.values():
    protein_ids.extend(value.keys())

#clean the list
protein_ids = list(set(protein_ids))
protein_ids = [id for id in protein_ids if id != '']


In [ ]:
print(protein_ids)
print(len(protein_ids))

['116292172', '78486550', '2507196', '2853980', '32425330', '115347926', '5454102', '148378801', '2393947', '6680530', '190938', '180352', '21595511', '15610402', '9629361', '597517618', '68989256', '4503385', '536029', '83627717', '9955963', '21392848', '12644416', '6831552', '118764400', '81899072', '119622516', '339641', '285809906', '735367775', '14149746', '1655766739', '20072248', '15610945', '31881630', '63102437', '341916350', '12803275', '116076351', '68474550', '1762973', '41872631', '56202836', '4506243', '4758484', '21264324', '119580345', '85666113', '76496497', '119607129', '7108463', '47123300', '1628587', '88702791', '4506055', '7706135', '493539358', '27894344', '67191027', '67463989', '134142337', '115430235', '208342286', '47678551', '60391226', '1654220559', '149631', '23110962', '1730321', '166202459', '86990435', '4826834', '4758878', '38027923', '4502331', '1927', '23893668', '998701', '27368096', '23943882', '110611243', '21595776', '28373962', '37589898', '5641

Now we retrieve all the FASTA sequences of proteins tested for all of our compounds with Biopython API to Entrez of NCBI. The FASTA sequence is saved as "sequences.fasta"

In [ ]:
# Always tell NCBI who you are
Entrez.email = "hdong26@amherst.edu"

# The filename where you want to save the sequences
output_filename = f'{data_folder}/before_finished/step_11/11_1/sequences.fasta'

# Open a file to write the sequences
with open(output_filename, "w") as output_file:
    for id in protein_ids:
        try:
            # Fetch the sequence from NCBI
            handle = Entrez.efetch(db="protein", id=id, rettype="fasta", retmode="text")
            sequence_data = handle.read()
            handle.close()

            # Write the sequence data to the file
            output_file.write(sequence_data)
        except Exception as e:
            print(f"An error occurred while fetching {id}: {e}")


From the FASTA sequence, we also need to retrieve the list of protein names, since these are different from the protein GIs

In [ ]:
def extract_protein_names(file_path):
    protein_names = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('>'):
                # Split the line at spaces and take the first item
                parts = line.split(' ')
                protein_name = parts[0]
                # Remove the leading '>' character
                protein_name = protein_name[1:]
                protein_names.append(protein_name)
    return protein_names

In [ ]:
file_path = f'{data_folder}/before_finished/step_11/11_1/sequences.fasta'
protein_names = extract_protein_names(file_path)
print(protein_names)

['NP_001013728.2', 'NP_598230.2', 'sp|P53779.2|MK10_HUMAN', 'AAC83551.1', 'AAH19268.2', 'CAL20848.1', 'NP_006333.1', 'YP_001253342.1', 'AAC51766.1', 'NP_032451.1', 'AAA36557.1', 'AAA51985.1', 'AAH32261.1', 'NP_217783.1', 'NP_057851.1', 'CDO16711.1', 'NP_004283.2', 'NP_000786.1', 'CAA84846.1', 'NP_031943.3', 'NP_005680.1', 'NP_652928.1', 'sp|Q14749.3|GNMT_HUMAN', 'sp|O75388.1|GPR32_HUMAN', 'AAI28575.1', 'sp|Q8C6L5.1|CGAS_MOUSE', 'EAX02111.1', 'AAC63054.1', 'DAA06693.1', 'NP_000202.3', 'NP_066285.1', 'NP_001357589.1', 'AAH26347.1', 'NP_218326.1', 'NP_000947.2', 'AAH95408.1', 'AAD14062.3', 'AAH02453.1', 'ABJ54071.1', 'XP_718648.1', 'AAC50996.1', 'NP_004095.4', 'CAI19851.1', 'NP_002810.1', 'NP_004823.1', 'NP_612200.1', 'EAW59941.1', 'NP_009905.3', 'NP_001029027.1', 'EAW86723.1', 'AAC50179.2', 'AAH70052.1', 'AAB17381.1', 'NP_002384.2', 'NP_002721.1', 'NP_057685.1', 'WP_006493245.1', 'NP_775180.1', 'NP_000876.3', 'pdb|1ZHH|B', 'NP_004987.2', 'NP_001041666.1', 'ACI25593.1', 'CAG30396.1', 'sp|

In [ ]:
# Create a dictionary to map protein IDs to protein names by index
protein_id_to_name = {protein_ids[i]: protein_names[i] for i in range(len(protein_ids))}

In [ ]:
print(protein_id_to_name)

{'116292172': 'NP_001013728.2', '78486550': 'NP_598230.2', '2507196': 'sp|P53779.2|MK10_HUMAN', '2853980': 'AAC83551.1', '32425330': 'AAH19268.2', '115347926': 'CAL20848.1', '5454102': 'NP_006333.1', '148378801': 'YP_001253342.1', '2393947': 'AAC51766.1', '6680530': 'NP_032451.1', '190938': 'AAA36557.1', '180352': 'AAA51985.1', '21595511': 'AAH32261.1', '15610402': 'NP_217783.1', '9629361': 'NP_057851.1', '597517618': 'CDO16711.1', '68989256': 'NP_004283.2', '4503385': 'NP_000786.1', '536029': 'CAA84846.1', '83627717': 'NP_031943.3', '9955963': 'NP_005680.1', '21392848': 'NP_652928.1', '12644416': 'sp|Q14749.3|GNMT_HUMAN', '6831552': 'sp|O75388.1|GPR32_HUMAN', '118764400': 'AAI28575.1', '81899072': 'sp|Q8C6L5.1|CGAS_MOUSE', '119622516': 'EAX02111.1', '339641': 'AAC63054.1', '285809906': 'DAA06693.1', '735367775': 'NP_000202.3', '14149746': 'NP_066285.1', '1655766739': 'NP_001357589.1', '20072248': 'AAH26347.1', '15610945': 'NP_218326.1', '31881630': 'NP_000947.2', '63102437': 'AAH95408

### 11.1.3 Percent Sequence Identity by Multiple Sequence Alignment

The sequences.fasta file is submitted to https://www.ebi.ac.uk/jdispatcher/msa/clustalo for multiple sequencing alignment. The resulted table of percent sequence identity matrix is saved and imported for the calculation of FoH

In [ ]:
#import the identity matrix:
protein_si = pd.read_csv(f'{data_folder}/before_finished/step_11/11_1/percent_identity_matrix.txt', delimiter='\s+', header=None)
# Modify the file name

In [ ]:
#remove the first column:
protein_si = protein_si.drop(protein_si.columns[0], axis=1)

In [ ]:
name = protein_si[1].tolist()
name = ['protein name'] + name

In [ ]:
#rename the columns in protein_si by the list name
protein_si.columns = name

In [ ]:
protein_si

,protein name,AAH65243.1,NP_001341533.1,NP_937802.1,AAA34498.1,pdb|1ULL|B,AAA36557.1,AAB23646.1,NP_000469.3,NP_001074551.1,...,NP_000762.2,NP_001970.2,NP_058021.1,XP_001348533.1,sp|P08753.3|GNAI3_RAT,NP_066268.1,NP_057231.1,YP_001253342.1,XP_718648.1,AAF25870.1
0,AAH65243.1,100.00,97.37,97.34,13.33,23.08,10.53,33.33,12.38,14.29,...,14.63,10.14,16.36,6.45,5.66,7.41,8.79,13.01,2.50,3.12
1,NP_001341533.1,97.37,100.00,100.00,13.33,23.08,10.53,33.33,10.64,14.97,...,13.56,9.17,17.91,4.96,7.95,7.87,10.99,12.31,8.47,7.38
2,NP_937802.1,97.34,100.00,100.00,13.33,23.08,10.53,33.33,10.64,14.97,...,13.91,9.40,17.91,5.08,8.24,8.14,10.99,12.60,8.62,7.44
3,AAA34498.1,13.33,13.33,13.33,100.00,17.65,21.05,25.00,11.11,11.11,...,NaN,0.00,10.00,0.00,0.00,0.00,6.25,11.11,NaN,0.00
4,pdb|1ULL|B,23.08,23.08,23.08,17.65,100.00,17.65,0.00,0.00,0.00,...,NaN,0.00,11.11,0.00,0.00,0.00,10.00,8.33,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,NP_066268.1,7.41,7.87,8.14,0.00,0.00,8.33,11.11,9.52,9.64,...,13.40,14.49,11.98,23.05,71.39,100.00,12.00,16.97,10.43,9.35
421,NP_057231.1,8.79,10.99,10.99,6.25,10.00,10.00,9.09,13.00,13.51,...,13.43,16.13,13.98,7.04,18.00,12.00,100.00,19.45,16.38,10.75
422,YP_001253342.1,13.01,12.31,12.60,11.11,8.33,14.81,7.14,10.85,12.33,...,19.29,14.73,20.22,17.55,18.05,16.97,19.45,100.00,19.00,17.72
423,XP_718648.1,2.50,8.47,8.62,NaN,NaN,0.00,NaN,15.07,13.70,...,11.72,10.62,12.02,13.85,10.95,10.43,16.38,19.00,100.00,19.34


### 11.1.4 Calculation of FoHs:

Until now, we have a dictionary of (cid: assays tested); (assay_tested:protein name), and percentage identity matrix with first columns as protein names.
For each compound, we retrieve the list of all protein names tested on that compounds by matching between the two first dictionary. From this list, we retrieve the corresponding matrix of percentages identitiy of these proteins corresponding to these compounds and calculate the FoH

In [ ]:
protein_si_dict = {}
for name in protein_si['protein name']:
    for other_name in protein_si['protein name']:
        if other_name != name:
            protein_si_dict[(name, other_name)] = protein_si.loc[protein_si['protein name'] == name, other_name].values[0]

In [ ]:
foh_dict = {}

for cid, targets in tqdm.tqdm(results_dict.items()):
    active_weight_list = []
    total_weight_list = []

    for target_id, result in targets.items():
        if target_id == '':
            continue

        protein_name = protein_id_to_name[target_id]
        max_weight = 0

        for other_id, other_result in targets.items():
            if other_id != target_id and other_id != '':
                other_protein_name = protein_id_to_name[other_id]
                value = protein_si_dict[(protein_name, other_protein_name)]
                max_weight = max(max_weight, value)

        target_weight = 1 - max_weight / 100

        if result:
            active_weight_list.append(target_weight)
        total_weight_list.append(target_weight)

    if total_weight_list:
        foh_score = sum(active_weight_list) / sum(total_weight_list)
        foh_dict[cid] = foh_score
    else:
        foh_dict[cid] = 0


100%|██████████| 188/188 [00:22<00:00,  8.39it/s]


In [ ]:
#export foh_dict
with open(f'{data_folder}/before_finished/step_11/11_1/foh_dict.json', 'w') as f:
    json.dump(foh_dict, f)

For compounds with FoH larger than 0.26, we remove them

In [ ]:
to_drop = []
for cid, foh_score in foh_dict.items():
    if foh_score > 0.26:
        to_drop.append(cid)

post_FoH_hits = pre11_hits[~pre11_hits['PUBCHEM_CID'].isin(to_drop)]
print(f'Dropped {(len(to_drop))} compounds with FoH larger than 0.26')

Dropped 1 compounds with FoH larger than 0.26


In [ ]:
# Save post_FoH_hits:
post_FoH_hits.to_csv(f'{data_folder}/before_finished/step_11/11_1/post_FoH_hits.csv', index=False)

In [ ]:
pre11_inactives.to_csv(f'{data_folder}/before_finished/step_11/11_1/post_FoH_inactives.csv', index=False)

## 11.2 Autofluoresence & Luceferase inhibition

When finding false positive due to autofluorescence and luceferase inhibition, it is important to check if the particular assays use one of these technologies. Here, all three assays (AID626, AID1488, and AID1741) use fluorescence technologies, so it is optimal to remove compounds that are active in AIDs: 587, 588, 590, 591, 592, 593, 594

In [ ]:
autofluorescence_aids = ['587', '588', '590', '591', '592', '593', '594']
autofluorescence_cids = []
col_list = ['PUBCHEM_CID', 'PUBCHEM_ACTIVITY_OUTCOME']

count = 0
for AID in autofluorescence_aids:
    url = f'https://pubchem.ncbi.nlm.nih.gov/assay/pcget.cgi?query=download&record_type=datatable&actvty=all&response_type=save&aid={AID}'
    autofluorescence_df = pd.read_csv(url, usecols=col_list)
    #delete rows with nan values
    autofluorescence_df = autofluorescence_df.dropna(subset=['PUBCHEM_CID', 'PUBCHEM_ACTIVITY_OUTCOME'])

    #convert cids to int:
    autofluorescence_df['PUBCHEM_CID'] = autofluorescence_df['PUBCHEM_CID'].astype(int)

    #keep only rows said "Active"
    autofluorescence_df = autofluorescence_df[autofluorescence_df['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
    autofluorescence_cids.extend(autofluorescence_df['PUBCHEM_CID'].tolist())


In [ ]:
#drop duplicates in the list:
autofluorescence_cids = [item for item in set(autofluorescence_cids)]

In [ ]:
to_drop_hits = []
for cid in post_FoH_hits:
    if cid in autofluorescence_cids:
        to_drop.append(cid)
post_autofluorescence = post_FoH_hits[~post_FoH_hits['PUBCHEM_CID'].isin(to_drop_hits)]
print(f'Dropped {(len(to_drop_hits))} autofluorescence compounds')

Dropped 0 autofluorescence compounds


In [ ]:
post_autofluorescence

,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,InChI,Mol removed from mixture,Small inorganic molecule
0,2882111,Cc1ccc(C)c(OCC(O)CN2C(C)CCCC2C)c1,Active,InChI=1S/C18H29NO2/c1-13-8-9-14(2)18(10-13)21-...,Cl,Cl
1,2328849,CCOC(=O)N1CCN(C2=Nc3cccc4cccc2c34)CC1,Active,InChI=1S/C18H19N3O2/c1-2-23-18(22)21-11-9-20(1...,NaN,NaN
2,2321890,CC(=O)Nc1ccc(S(=O)(=O)Nc2nc3ccccc3nc2Sc2nncn2C...,Active,InChI=1S/C19H17N7O3S2/c1-12(27)21-13-7-9-14(10...,NaN,NaN
3,2310918,C=CCn1c(=O)/c(=C/c2ccc(O)cc2)s/c1=C(/C#N)c1nnc...,Active,InChI=1S/C22H21N5O2S/c1-2-11-27-21(29)18(13-15...,NaN,NaN
4,2548564,CCN(CC(=O)Nc1ccc2c(c1)OCCO2)c1nc(-c2cccnc2)nc2...,Active,InChI=1S/C25H23N5O3/c1-2-30(16-23(31)27-18-9-1...,NaN,NaN
...,...,...,...,...,...,...
183,6602803,CCc1nc(NCc2ccccn2)c2oc3ccccc3c2n1,Active,InChI=1S/C18H16N4O/c1-2-15-21-16-13-8-3-4-9-14...,Cl,Cl
184,664440,CCOC(=O)c1cc2c(=O)n3cccc(C)c3nc2n(Cc2ccco2)c1=...,Active,InChI=1S/C26H21N5O5/c1-3-35-26(34)20-13-19-22(...,NaN,NaN
185,664704,Cc1cccn2c(=O)c3cc(C#N)c(=NC(=O)c4cccnc4)n(CC4C...,Active,InChI=1S/C24H20N6O3/c1-15-5-3-9-29-20(15)27-22...,NaN,NaN
186,664895,CC(C)OCCCNc1ncnc2c1[nH]c1ccc(F)cc12,Active,InChI=1S/C16H19FN4O/c1-10(2)22-7-3-6-18-16-15-...,NaN,NaN


In [ ]:
if not os.path.exists(f'{data_folder}/before_finished/step_11/11_2'):
    os.makedirs(f'{data_folder}/before_finished/step_11/11_2')

In [ ]:
#save:
post_autofluorescence.to_csv(f'{data_folder}/before_finished/step_11/11_2/post_autofluorescence_hits.csv', index=False)
pre11_inactives.to_csv(f'{data_folder}/before_finished/step_11/11_2/post_autofluorescence_inactives.csv', index=False)

## 11.3 RDKit PAIN filter

In [ ]:
params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS)
catalog = FilterCatalog(params)

In [ ]:
def detect_pains(df):
    pains = []
    count_pains = 0
    count_not_pains = 0
    smiles_column = 'PUBCHEM_EXT_DATASOURCE_SMILES'
    cids_column = 'PUBCHEM_CID'
    for i in df.index:
        smile = str(df.loc[i, smiles_column])
        cid = df.loc[i, cids_column]
        mol = Chem.MolFromSmiles(smile)
        if mol is not None:
            if catalog.HasMatch(mol):
                pains.append(cid)
                count_pains += 1
                print(f'{count_pains} pains detected')
            else:
                count_not_pains += 1
    return pains

pains = detect_pains(post_autofluorescence)

1 pains detected
2 pains detected
3 pains detected
4 pains detected
5 pains detected
6 pains detected
7 pains detected
8 pains detected
9 pains detected
10 pains detected
11 pains detected
12 pains detected
13 pains detected
14 pains detected
15 pains detected
16 pains detected
17 pains detected
18 pains detected


In [ ]:
post_pains_hits = post_autofluorescence[~post_autofluorescence['PUBCHEM_CID'].isin(pains)]

In [ ]:
if not os.path.exists(f'{data_folder}/before_finished/step_11/11_3'):
    os.makedirs(f'{data_folder}/before_finished/step_11/11_3')

post_pains_hits.to_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_hits.csv', index=False)
pre11_inactives.to_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_inactives.csv', index=False)

# 12. Drug-likeness filter:

In [ ]:
pre12_hits = pd.read_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_hits.csv', sep=',', header=0)
pre12_inactives = pd.read_csv(f'{data_folder}/before_finished/step_11/11_3/post_pains_inactives.csv', sep=',', header=0)

In [ ]:
def drug_likeness_filter(smiles):
    """
    This functions check if a given smiles satisfies the common standard conditions for drug-likeness.
    Input:
        SMILES (str)
    Output:
        Result (bool)
    """

    # Convert SMILES string to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return False  # Return False if the molecule cannot be parsed

    # Check molecular weight
    mw = Chem.rdMolDescriptors.CalcExactMolWt(mol)
    if not (150 < mw < 800):
        return False

    # Check AlogP
    logp = Chem.Crippen.MolLogP(mol)
    if not (-0.3 < logp < 5):
        return False

    # Check number of rotatable bonds
    rotatable_bonds = Lipinski.NumRotatableBonds(mol)
    if rotatable_bonds >= 15:
        return False

    # Check H-bond acceptor count and H-bond donor count
    hba = Lipinski.NumHAcceptors(mol)
    hbd = Lipinski.NumHDonors(mol)
    if hba >= 15 or hbd >= 15:
        return False

    # Check total formal charge
    total_charge = sum(atom.GetFormalCharge() for atom in mol.GetAtoms())
    if not (-2 < total_charge < 2):
        return False

    # If all filters passed, return True
    return True

def drug_likeness_filter_multiprocessing(df):
    """
    This function update a given dataframe by dropping molecules that don't pass the drug-likeness filter.
    """
    to_drop = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Create future tasks for each SMILES string in the dataframe
        futures = {executor.submit(drug_likeness_filter, row['PUBCHEM_EXT_DATASOURCE_SMILES']): row['PUBCHEM_CID'] for index, row in df.iterrows()}

        # Use tqdm to display progress bar
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing SMILES"):
            cid = futures[future]
            if not future.result():
                to_drop.append(cid)
    return to_drop

In [ ]:
not_drug_hits = drug_likeness_filter_multiprocessing(pre12_hits)

Processing SMILES: 100%|██████████| 169/169 [00:00<?, ?it/s]


In [ ]:
not_drug_inactives = drug_likeness_filter_multiprocessing(pre12_inactives)

Processing SMILES: 100%|██████████| 61725/61725 [00:10<00:00, 5705.98it/s]  


In [ ]:
post12_hits = pre12_hits[~pre12_hits['PUBCHEM_CID'].isin(not_drug_hits)]
post12_inactives = pre12_inactives[~pre12_inactives['PUBCHEM_CID'].isin(not_drug_inactives)]

print(f'Dropped {(len(not_drug_hits))} hit compounds that do not pass the drug likeness filter')
print(f'Dropped {(len(not_drug_inactives))} inactive compounds that do not pass the drug likeness filter')


Dropped 4 hit compounds that do not pass the drug likeness filter
Dropped 1169 decoy compounds that do not pass the drug likeness filter


In [ ]:
#Export not_drug_hits and inactives:
with open(f'{data_folder}/before_finished/step_12/not_drug_hits.json', 'w') as f:
    json.dump(not_drug_hits, f)
with open(f'{data_folder}/before_finished/step_12/not_drug_inactives.json', 'w') as f:
    json.dump(not_drug_inactives, f)

In [ ]:
# save:
if not os.path.exists(f'{data_folder}/before_finished/step_12'):
    os.makedirs(f'{data_folder}/before_finished/step_12')

post12_hits.to_csv(f'{data_folder}/before_finished/step_12/post12_hits.csv', index=False)
post12_inactives.to_csv(f'{data_folder}/before_finished/step_12\post12_inactives.csv', index=False)

# 13. ChemBL Curation Pipeline

In [ ]:
pre13_hits = pd.read_csv(f'{data_folder}/before_finished/step_12/post12_hits.csv', sep=',', header=0)
pre13_inactives = pd.read_csv(f'{data_folder}/before_finished/step_12/post12_inactives.csv', sep=',', header=0)

In [ ]:
def checker_score(smiles, cid):
    result = checker.check_molblock(Chem.MolToMolBlock(Chem.MolFromSmiles(smiles)))
    if result == ():
        penalty_score = 0
    else:
        penalty_score = result[0][0]
    return cid, penalty_score

def checker_multiprocessing(df):
    chembl_score_dict = {}
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Map futures to CIDs directly for easier reference
        futures = {executor.submit(checker_score, row['PUBCHEM_EXT_DATASOURCE_SMILES'], row['PUBCHEM_CID']): row['PUBCHEM_CID'] for _, row in df.iterrows()}
        # Properly use tqdm to create a progress bar
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing SMILES"):
            cid, penalty_score = future.result()
            chembl_score_dict[cid] = penalty_score
    return chembl_score_dict

In [ ]:
score_hits = checker_multiprocessing(pre13_hits)
score_inactives = checker_multiprocessing(pre13_inactives)

Processing SMILES: 100%|██████████| 60556/60556 [00:42<00:00, 1412.51it/s] 


In [ ]:
#print all unique values in the dictionary:
print(set(score_hits.values()))
print(set(score_inactives.values()))

{0, 2}
{0, 2, 5, 6}


In [ ]:
# Create a new step folder
if not os.path.exists(f'{data_folder}/before_finished/step_13'):
    os.makedirs(f'{data_folder}/before_finished/step_13')

# save the scores:
with open(f'{data_folder}/before_finished/step_13/score_hits.json', 'w') as f:
    json.dump(score_hits, f)
with open(f'{data_folder}/before_finished/step_13/score_inactives.json', 'w') as f:
    json.dump(score_inactives, f)

In [ ]:
#drop all compounds with a penalty score of 7:
to_drop_hits = []
to_drop_inactives = []
for cid, penalty_score in score_hits.items():
    if penalty_score == 7:
        to_drop_hits.append(cid)
for cid, penalty_score in score_inactives.items():
    if penalty_score == 7:
        to_drop_inactives.append(cid)

post13_hits = pre13_hits[~pre13_hits['PUBCHEM_CID'].isin(to_drop_hits)]
post13_inactives = pre13_inactives[~pre13_inactives['PUBCHEM_CID'].isin(to_drop_inactives)]

In [ ]:
#save final_hits and inactives:
post13_hits.to_csv(f'{data_folder}/before_finished/step_13/post13_hits.csv', index=False)
post13_inactives.to_csv(f'{data_folder}/before_finished/step_13/post13_inactives.csv', index=False)

# 14. Final handling of chemical representation

There are two problems that requires InChI update:

(1) Some of the InChI will be missing, since the PubChem Identifier Exchange service might not able to find the corresponding InChI for the aromatized, neutralized SMILES.

(2) While handling mixtures, some mixtures whose component molecules are identical will result in duplicates. Therefore, we need to check their activities.
- If all duplicates share the same results (active/inactive), we keep one of them.
- If duplicates of the same molecules returned different activity, we remove both of them.

In [ ]:
#import:
pre14_hits = pd.read_csv(f'{data_folder}/before_finished/step_13/post13_hits.csv', sep=',', header=0)
pre14_inactives = pd.read_csv(f'{data_folder}/before_finished/step_13/post13_inactives.csv', sep=',', header=0)

## 14.1 Update InChI

In [ ]:
def smi_to_inchi(smi):
    mol = Chem.MolFromSmiles(smi)
    inchi = Chem.inchi.MolToInchi(mol)
    return inchi

In [ ]:
count = 0
for index, row in pre14_hits.iterrows():
    if row['InChI'] != row['InChI']:
        pre14_hits.at[index, 'InChI'] = smi_to_inchi(row['PUBCHEM_EXT_DATASOURCE_SMILES'])
        count += 1
print(f'Updated {count} InChI values in pre14_hits')

count = 0
for index, row in pre14_inactives.iterrows():
    if row['InChI'] != row['InChI']:
        pre14_inactives.at[index, 'InChI'] = smi_to_inchi(row['PUBCHEM_EXT_DATASOURCE_SMILES'])
        count += 1
print(f'Updated {count} InChI values in pre14_inactives')

Updated 0 InChI values in pre13_hits


[14:56:23] WARNING: Charges were rearranged

[14:56:24] WARNING: Omitted undefined stereo

[14:56:24] WARNING: Omitted undefined stereo

[14:56:24] WARNING: Omitted undefined stereo

[14:56:24] WARNING: Omitted undefined stereo

[14:56:25] WARNING: Omitted undefined stereo

[14:56:25] WARNING: Omitted undefined stereo



Updated 25 InChI values in pre13_decoys


[14:56:25] WARNING: Omitted undefined stereo



## 14.2 Handle duplicates

In [ ]:
#Check if a mol in hit set appeared in inactive set:
for i in pre14_hits['PUBCHEM_EXT_DATASOURCE_SMILES']:
    if i in list(pre14_inactives['PUBCHEM_EXT_DATASOURCE_SMILES']):
        print('{i} SMILES appeared in both hit and inactive sets')
for i in pre14_hits['InChI']:
    if i in list(pre14_inactives['InChI']):
        print('{i} InChI appeared in both hit and inactive sets')

#Return all duplicates by comparing InChI:
final_hits_duplicates_InChI = pre14_hits[pre14_hits.duplicated(subset=['InChI'], keep=False)]
final_inactives_duplicates_InChI = pre14_inactives[pre14_inactives.duplicated(subset=['InChI'], keep=False)]
final_hits_duplicates_smi = pre14_hits[pre14_hits.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]
final_inactives_duplicates_smi = pre14_inactives[pre14_inactives.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]

print('Number of InChI duplicates in hits: ', len(final_hits_duplicates_InChI))
print('Number of InChI duplicates in inactives: ', len(final_inactives_duplicates_InChI))
print('Number of SMILES duplicates in hits: ', len(final_hits_duplicates_smi))
print('Number of SMILES duplicates in inactives: ', len(final_inactives_duplicates_smi))

Number of InChI duplicates in hits:  2
Number of InChI duplicates in decoys:  25
Number of SMILES duplicates in hits:  2
Number of SMILES duplicates in decoys:  25


In [ ]:
if not os.path.exists(f'{data_folder}/before_finished/step_14'):
    os.makedirs(f'{data_folder}/before_finished/step_14')

#write all the duplicates to a file:
#write duplicates to a txt file:
with open(f'{data_folder}/before_finished/step_14/duplicates.txt', 'w') as f:
    f.write('InChI duplicates in hits: \n')
    f.write(final_hits_duplicates_InChI.to_string())
    f.write('\n\n')
    f.write('InChI duplicates in inactives: \n')
    f.write(final_inactives_duplicates_InChI.to_string())
    f.write('\n\n')
    f.write('SMILES duplicates in hits: \n')
    f.write(final_hits_duplicates_smi.to_string())
    f.write('\n\n')
    f.write('SMILES duplicates in inactives: \n')
    f.write(final_inactives_duplicates_smi.to_string())

In [ ]:
#remove these duplicates, keep the first one:
#by inchi:
final_hits = pre14_hits.drop_duplicates(subset=['InChI'], keep='first')
final_inactives = pre14_inactives.drop_duplicates(subset=['InChI'], keep='first')

In [ ]:
#as expected, there might still be SMILES duplicates:
if len(final_hits[final_hits.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]) == 0:
    print('No more duplicates in hits')

if len(final_inactives[final_inactives.duplicated(subset=['PUBCHEM_EXT_DATASOURCE_SMILES'], keep=False)]) == 0:
    print('No more duplicates in inactives')

No more duplicates in hits
No more duplicates in decoys


In [ ]:
# save:
final_hits.to_csv(f'{data_folder}/finished/final_hits.csv',sep=';', index=False)
final_inactives.to_csv(f'{data_folder}/finished/final_inactives.csv',sep=';', index=False)

# 0.1 Adjusting Columns

In [ ]:
final_hits = pd.read_csv(f'{data_folder}/finished/final_hits.csv', sep=',', header=0)
final_inactives = pd.read_csv(f'{data_folder}/finished/final_inactives.csv', sep=',', header=0)

In [ ]:
# Add another column: "activity_value" with all empty NaN values:
final_hits.loc[:, 'activity_value'] = np.nan
final_inactives.loc[:, 'activity_value'] = np.nan

In [ ]:
# Rename the columns:
final_hits = final_hits.rename(columns={
    'PUBCHEM_CID': 'CID',
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
    'Mol removed from mixture': 'mol_removed_from_mixture',
    'Small inorganic molecule': 'small_inorganic_mol_from_mixture',
    'Small organic molecule': 'small_organic_mol_from_mixture'
})
final_inactives = final_inactives.rename(columns={
    'PUBCHEM_CID': 'CID',
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
    'Mol removed from mixture': 'mol_removed_from_mixture',
    'Small inorganic molecule': 'small_inorganic_mol_from_mixture',
    'Small organic molecule': 'small_organic_mol_from_mixture'
})

In [ ]:
# If there is no "small_organic_mol_from_mixture" column, add it:
if 'small_organic_mol_from_mixture' not in final_hits.columns:
    final_hits.loc[:, 'small_organic_mol_from_mixture'] = np.nan
if 'small_organic_mol_from_mixture' not in final_inactives.columns:
    final_inactives.loc[:, 'small_organic_mol_from_mixture'] = np.nan

In [ ]:
#swap the positions of the columns InChI and activity_outcome:
final_hits = final_hits[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]
final_inactives = final_inactives[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]

In [ ]:
#export:
final_hits.to_csv(f'{data_folder}/finished/final_hits.csv', sep=',', index=False)
final_inactives.to_csv(f'{data_folder}/finished/final_inactives.csv', sep=',', index=False)

# 0.2 Convert to txt for CORINA

In [ ]:
# import the final hits and inactives:
final_hits = pd.read_csv(f'{data_folder}/finished/final_hits.csv', sep=',', header=0)
final_inactives = pd.read_csv(f'{data_folder}/finished/final_inactives.csv', sep=',', header=0)

# export to .txt files:
final_hits.to_csv(f'{data_folder}/finished/final_hits.txt', sep=';', index=False, header=False)
final_inactives.to_csv(f'{data_folder}/finished/final_inactives.txt', sep=';', index=False, header=False)

# 0.3. Exporting raw data without further curation (only smiles, inchi) for control experiments

In [ ]:
raw_M1_agonist = pd.read_csv(f'{data_folder}/before_finished/step_1/AID626.csv', sep=',', header=0)

In [ ]:
std_inchi = pd.read_csv(f'{data_folder}/before_finished/step_3/std_inchi_626.txt', sep='\t', header=None)

In [ ]:
#Convert AID626_InChI to a dictionary:
raw_M1_agonist_dict = dict(zip(std_inchi[0], std_inchi[1]))
#Add InChI to AID626 dataframe:
raw_M1_agonist['InChI'] = raw_M1_agonist['PUBCHEM_CID'].map(raw_M1_agonist_dict)

In [ ]:
raw_hits = raw_M1_agonist[raw_M1_agonist['PUBCHEM_ACTIVITY_OUTCOME'] == 'Active']
raw_inactives = raw_M1_agonist[raw_M1_agonist['PUBCHEM_ACTIVITY_OUTCOME'] == 'Inactive']

In [ ]:
# Add some other columns to match the format of the curated data:
raw_hits.loc[:, 'activity_value'] = np.nan
raw_hits.loc[:, 'mol_removed_from_mixture'] = np.nan
raw_hits.loc[:, 'small_inorganic_mol_from_mixture'] = np.nan
raw_hits.loc[:, 'small_organic_mol_from_mixture'] = np.nan

raw_inactives.loc[:, 'activity_value'] = np.nan
raw_inactives.loc[:, 'mol_removed_from_mixture'] = np.nan
raw_inactives.loc[:, 'small_inorganic_mol_from_mixture'] = np.nan
raw_inactives.loc[:, 'small_organic_mol_from_mixture'] = np.nan

In [ ]:
# Rename the columns:
raw_hits = raw_hits.rename(columns={
    'PUBCHEM_CID': 'CID',
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
})
raw_inactives = raw_inactives.rename(columns={
    'PUBCHEM_CID': 'CID',
    'PUBCHEM_ACTIVITY_OUTCOME': 'activity_outcome',
    'PUBCHEM_EXT_DATASOURCE_SMILES': 'SMILES',
})

In [ ]:
#swap the positions of the columns InChI and activity_outcome:
raw_hits = raw_hits[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]
raw_inactives = raw_inactives[['CID', 'SMILES', 'InChI', 'activity_outcome', 'activity_value', 'mol_removed_from_mixture', 'small_inorganic_mol_from_mixture', 'small_organic_mol_from_mixture']]

In [ ]:
if not os.path.exists(f'{data_folder}/finished/control_data'):
    os.makedirs(f'{data_folder}/finished/control_data')

#save the hits and inactives
raw_hits.to_csv(f'{data_folder}/finished/control_data/raw_hits.csv', sep=',', index=False)
raw_inactives.to_csv(f'{data_folder}/finished/control_data/raw_inactives.csv', sep=',', index=False)

#save as txt:
raw_hits.to_csv(f'{data_folder}/finished/control_data/raw_hits.txt', sep=';', index=False, header=False)
raw_inactives.to_csv(f'{data_folder}/finished/control_data/raw_inactives.txt', sep=';', index=False, header=False)